In [2]:
# preamble
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp

# Optimising speech recognition dataset partitions utilising linear programming

Traditionally speech datasets split into training, development and test sets are generally constrained by:
* Ensuring there is no overlap of speakers across the sets, i.e. that the speakers are unique within each partition.
* The duration of the training, development and test sets are rougly 80%, 10%, 10% respectively.
* Finally, if possible, ensuring that there is an equal balance between male and female speakers in the sets.

However, partitioning the corpus under investigation is complicated by the diverse variety of language combinations utttered by each speaker.
Our considered dataset consists of 50 hours of code-switched speech in eight Southern African languages (English, isiZulu, isiXhosa, Sesotho, Setswana, Sepedi, Tsotsitaal, and Afrikaans) from 307 different speakers.

Intrasentential code-switching, which is the use of multiple languages within sentences, is a common occurance in this dataset.
Because the primary focus of our dataset is to evaluate speech recognition systems on code-switched speech, this data should be fairly represented in the test partition.
However, partitioning the corpus under investigation is complicated by the diverse variety of language combinations uttered by each speaker.
In total, 94 different combinations of one, two, three or even four of the the eight languages present in the dataset were observed in single utterances.
Additionally, we observe that a large proportion of the speakers produce solely monolingual speech, while utterances by other speakers are a mix of code-switched and monolingual speech.

Partitioning this dataset is a complex combinatorial problem because 307 speakers can be assigned to three different partitions in $3^{307} - 3 \cdot 2 ^{307} + 3$ different arrangements ([solution](https://math.stackexchange.com/questions/527216/distributing-n-different-things-among-r-distinct-groups-such-that-all-of-the)).

### We begin by loading in the dataset:
* Again, our dataset consists of speech in eight languages where up to four languages are used in intra-sentential code-switching and consists of 307 speakers.
* As evidenced in the dataframe, 94 distinct combinations of languages are present within utterances.

In [3]:
df = pd.read_csv("speech_per_speaker_per_language.csv")

df.head()

,speaker,AFR,ENG,FLY,NSO,SOT,TSN,XHO,ZUL,AFR-ENG,...,ENG-FLY-SOT-TSN,ENG-FLY-SOT-ZUL,ENG-FLY-TSN-ZUL,ENG-NSO-SOT-TSN,ENG-NSO-SOT-ZUL,ENG-NSO-TSN-ZUL,ENG-SOT-TSN-XHO,ENG-SOT-TSN-ZUL,ENG-SOT-XHO-ZUL,FLY-SOT-TSN-ZUL
0,SPEAKER_1,0.0,15.658,0.0,0.0,1.683,42.902,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SPEAKER_2,0.0,19.987,0.0,0.0,0.000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SPEAKER_3,0.0,6.969,0.0,0.0,0.000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SPEAKER_4,0.0,36.565,0.0,0.0,0.000,0.000,0.000,42.639,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SPEAKER_5,0.0,1431.205,0.0,0.0,110.253,43.627,0.563,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 95 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   speaker          307 non-null    object 
 1   AFR              307 non-null    float64
 2   ENG              307 non-null    float64
 3   FLY              307 non-null    float64
 4   NSO              307 non-null    float64
 5   SOT              307 non-null    float64
 6   TSN              307 non-null    float64
 7   XHO              307 non-null    float64
 8   ZUL              307 non-null    float64
 9   AFR-ENG          307 non-null    float64
 10  AFR-FLY          307 non-null    float64
 11  AFR-NSO          307 non-null    float64
 12  AFR-SOT          307 non-null    float64
 13  AFR-TSN          307 non-null    float64
 14  AFR-XHO          307 non-null    float64
 15  AFR-ZUL          307 non-null    float64
 16  ENG-FLY          307 non-null    float64
 17  ENG-NSO         

## Dataset paritioning as a resource allocation problem

Resource allocation problems are commonly addressed in research through linear programming, constraint optimisation, mixed integer programming, or probabilisitic graphical models.
This problem can be described as allocating $N_r$ resources or tasks ($r$) to $N_W$ workers ($w$) given a set of constraints. A cost matrix $\boldsymbol{C}$ of dimensionality $(N_w,N_r)$ defines a cost $c_{i,j}$ of assigning a worker $i$ to a specific resource $j$. Where each resource or task can additionally have a weight or time to complete $T$. 

We relate this problem by assigning $N_s$ speakers ($s=r$) to one of $N_p$ partitions ($p=w$), under certain constraints.

These constraints can be described as:
<ol>
  <li>Each speaker must be assigned to exactly one partition.</li>
  <li>The test set must contain at least 50 minutes of English-isiZulu, English-Sesotho, English-Setswana code-switched speech. And 35 minutes of English-isiXhosa code-switched speech.</li>
  <li>The development set must contain at least 15 minutes of English-isiZulu, English-isiXhosa, English-Sesotho, English-Setswana code-switched speech.</li>
  <li>Only code-switched utterances should be included in the test set, therefore monolingual speakers should only be included in training set.</li>
  <li>Speakers who contribute a large amount of monolingual speech or large amount of code-switched speech should be prioritised for assignment to the training set or the test set respectively.</li>
  <li>Require at least 16 speakers across English-isiZulu, English-isiXhosa, English-Sesotho, English-Setswana in the test set.</li>
  <li>Require at least 12 speakers across English-isiZulu, English-isiXhosa, English-Sesotho, English-Setswana in the development set.</li>
  <li>Speakers with utterances including code-switching between three or four languages as well as speakers who utter rare types of code-switching should be assigned to the test set with priority.</li>
</ol>

We utilise the following libraries:

* https://developers.google.com/optimization/bin/multiple_knapsack
* https://developers.google.com/optimization/assignment/overview

# Designing the cost matrix

In the following section we implement the constraints mentioned above (numbered) by imposing them either through the cost matrix $\boldsymbol{C}$ (constraints 4 and 5) or by adding constraints to the solver (constraints 1,2,3,6,7,8).

In [44]:
n_S = len(df) # number of speakers
n_P = 3 # number of partitions (train,dev,test)

np.random.seed(42) # random seed for reproducibility
C = np.random.uniform(0,0.1,(n_S,n_P)) # add some noise to avoid equal costs

### 4. Only code-switched utterances in test-set

i.e. exlude any only monolingual speakers from test set

In [45]:
non_mono_cols = list(df.columns)[9:] # first nine columns are all monolingual

# for all speakers create vector which assigns high cost of adding them to a specific set
mono_speaker = [10e6 if i else 0 for i in list(df[non_mono_cols].sum(axis=1) == 0)]

C[:,2] += mono_speaker # apply this cost to the test set
C[:,1] += mono_speaker # apply this cost to the dev set

### 4.* Maximise utterances in training set

Because the constraints for partitioning the set are chiefly aimed at including utterances in the development and test sets, we need to add a small cost of adding any utterance to development and test sets, otherwise no speakers will be assigned to training set.

In [46]:
C[:,1] += 100 # add small cost of assigning any speaker to development set
C[:,2] += 100 # add small cost of assigning any speaker to development set

### 5. Account cost for the amount of monolingual (not considering English) speech a speaker would contribute to the training set

And add the cost of assigning that to the development and test sets

In [47]:
mono_cols = list(df.columns)[1:9] # select all monolingual columns
mono_cols.remove('ENG') # remove English because its inclusion is dominating

mono_speech_per_speaker = df[mono_cols].sum(axis=1) # collect amount of monolingual speech each speaker contributed
all_mono_speech = mono_speech_per_speaker.sum() # collect amount of all monolingual speech

perc_mono_contribution = mono_speech_per_speaker / all_mono_speech * 100000 # convert amount to percentage and add weighting of 100000

C[:,1] += perc_mono_contribution # add cost of assiging speakers with a large amount of monolingual speech to development set
C[:,2] += perc_mono_contribution # add cost of assiging speakers with a large amount of monolingual speech to test set

#### 5.* Add additional cost for isiXhosa

Because we have a substantially smaller amount of ixiXhosa data, we include a further cost for assigning this monolingual speech to the test set.

In [48]:
# add cost of assiging speakers with a large amount of isiXhosa monolingual speech to development set
C[:,1] += df['XHO'] / all_mono_speech * 100000 * 4

# add cost of assiging speakers with a large amount of isiXhosa monolingual speech to test set
C[:,2] += df['XHO'] / all_mono_speech * 100000 * 4 

### 5. Account cost for the amount of CS speech a speaker would contribute to the dev/test set

And add the cost of assigning that to the training set

In [49]:
cs_cols = list(df.columns)[9:] # select all CS columns
cs_speech_per_speaker = df[cs_cols].sum(axis=1) # collect amount of CS speech each speaker contributed
all_cs_speech = cs_speech_per_speaker.sum()  # collect amount of all CS speech

perc_cs_contribution = cs_speech_per_speaker / all_cs_speech * 100 # convert amount to percentage and add weighting of 100

# add cost of assiging speakers with a large amount of CS speech to training set
# this is reduced because we do want some CS data in our training set
C[:,0] += perc_cs_contribution 

#### Show first 10 speakers

In [50]:
C[:10,:]

array([[9.90288963e-02, 2.00582888e+02, 2.00561016e+02],
       [5.98658484e-02, 1.00001000e+07, 1.00001000e+07],
       [5.80836122e-03, 1.00001001e+07, 1.00001001e+07],
       [7.21930183e-02, 1.96103886e+02, 1.96198819e+02],
       [4.20205800e-01, 4.53187957e+02, 4.53184906e+02],
       [1.83404510e-02, 1.00001023e+07, 1.00001023e+07],
       [4.31945019e-02, 1.00001000e+07, 1.00001001e+07],
       [4.47304426e-01, 2.03553251e+03, 2.03553993e+03],
       [4.94788134e-02, 1.00078518e+02, 1.00019967e+02],
       [1.41526981e-01, 1.06790442e+03, 1.06784983e+03]])

# Create solver

In [52]:
solver = pywraplp.Solver.CreateSolver('SCIP') # see https://www.scipopt.org for more details about this solver

# create dictionary (matrix) which denotes the possible states of the n_s and n_p allocation
x = {}
for i in range(n_S): # for each speaker
    for p in range(n_P): # for each parititon
        x[i, p] = solver.BoolVar(f'x_{i}_{p}') # create boolean variable (1 or 0)

### 1. Each speaker is assigned to only 1 partition. i.e. No speakers overlap in train, dev, and test

In [55]:
for i in range(n_S): # for each speaker
    # add constraint that each row in allocation matrix should sum to 1
    # for the boolean variable this ensures that only one value in row
    # can be 1 while other two columns are 0
    solver.Add(solver.Sum([x[i, j] for j in range(n_P)]) == 1)

### 2. Test set must contain at least 50 minutes of English-isiZulu, English-Sesotho, English-Setswana code-switched speech. And 35 minutes of English-isiXhosa code-switched speech.

In [61]:
minutes = 50 # minutes of speech for EZ,ES,ET
xho_minutes = 35 # minutes of speech for EX

# create empty matrix to store the amount of CS speech for each of the four
# considered language pairs (EZ,EX,ES,ET)
lang_ms = np.zeros((n_S,4),dtype=int)

# for each language pair
for j,lang in enumerate(['ENG-ZUL','ENG-SOT','ENG-TSN','ENG-XHO']):
    speech_s_per_speaker = df[lang] # amount of speech in lang pair for every speaker
    
    # for each speaker
    for i,spk in enumerate(speech_s_per_speaker):
        # add amount of CS speech to matrix
        lang_ms[i,j] += int(spk*1000)

j = 2 # test set

# for EZ,ES,ET
for lang in range(3):
    # add constraint that each column in allocation matrix should be larger
    # or equal to 60 minutes of speech (multiplication with lang_ms converts
    # indicator to amount of speech for the language pair for the speaker).
    solver.Add(solver.Sum([x[i,j] * lang_ms[i,lang] for i in range(n_S)]) >= minutes * 60 * 1000)

# require fewer minutes for isiXhosa - there is simply not enough EX code-switched speech to increase this value
lang = 3

# add constraint that each column in allocation matrix should be larger
# or equal to 35 minutes of speech (multiplication with lang_ms converts
# indicator to amount of speech for English-isiXhosa for the speaker).
solver.Add(solver.Sum([x[i,j] * lang_ms[i,lang] for i in range(n_S)]) >= xho_minutes * 60 * 1000)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fd2fb44ee10> >

### 3. Development set must contain at least 15 minutes of English-isiZulu, English-isiXhosa, English-Sesotho, English-Setswana code-switched speech.

In [62]:
j = 1 # development set

# for each language pair EZ,EX,ES,ET
for lang in range(4):
    # add constraint that each column in allocation matrix should be larger
    # or equal to 35 minutes of speech (multiplication with lang_ms converts
    # indicator to amount of speech for English-isiXhosa for the speaker).
    solver.Add(solver.Sum([x[i,j] * lang_ms[i,lang] for i in range(n_S)]) >= 900000)

### 6. Test paritition is assigned at least 16 speakers from English-isZulu, English-isiXhosa, English-Sesotho, and English-Setswana

In [56]:
lang = np.zeros((n_S,4)) # create matrix of zeros with n_s rows and four columns for each language pair

# for each language pair
for i,lp in enumerate(['ENG-ZUL','ENG-XHO','ENG-SOT','ENG-TSN']):
    indicator_vector = df[lp] > 0 # create indicator vector for all speakers who utter speech in that language pair
    lang[:,i] = indicator_vector # assign vector to language matrix

# for each language pair
for j in range(4):
    # add constraint that the sum of each column (number of speakers assigned to test set)
    # should be greater or equal to 16 for each language pair (element-wise multiplication
    # with lang vector selects only speakers who utterances contain the language pair).
    solver.Add(solver.Sum([x[i, 2] * lang[i,j] for i in range(n_S)]) >= 16)

### 7. Development partition is assigned atleast 12 speakers from English-isZulu, English-isiXhosa, English-Sesotho, and English-Setswana

In [16]:
# for each language pair
for j in range(4):
    # add constraint that the sum of each column (number of speakers assigned to development set)
    # should be greater or equal to 16 for each language pair (element-wise multiplication
    # with lang vector selects only speakers who utterances contain the language pair).
    solver.Add(solver.Sum([x[i, 1] * lang[i,j] for i in range(n_S)]) >= 12)

### 8. Assign rare forms of code-switching to test set

In [57]:
# create indicator vector for all speakers whose utterances contain code-switching between English and Sepedi
eng_nso_speaker = [1 if i else 0 for i in list(df['ENG-NSO'] > 0)]
# create indicator vector for all speakers whose utterances contain code-switching between English and Afrikaans
eng_afr_speaker = [1 if i else 0 for i in list(df['AFR-ENG'] > 0)]

all_nso_speakers = sum(eng_nso_speaker) # all speakers whose utterances contain code-switching between English and Sepedi
all_afr_speakers = sum(eng_afr_speaker)# all speakers whose utterances contain code-switching between English and Afrikaans

# add constraint that at least half of the speakers whose utterances contain code-switching between English and Sepedi
# and English and Afrikaans should be assigned to test set
solver.Add(solver.Sum([x[i, 2] * eng_nso_speaker[i] for i in range(n_S)]) >= all_nso_speakers // 2)
solver.Add(solver.Sum([x[i, 2] * eng_afr_speaker[i] for i in range(n_S)]) >= all_afr_speakers // 2)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fd2fb8156c0> >

## Define objective function and solve

In [63]:
# create list of linear objectives
objective_terms = []

for i in range(n_S): # for each speaker
    for j in range(n_P): # for each partition
        # add objective of allocation scalar to cost matrix 
        objective_terms.append(C[i][j] * x[i, j])

# set solver to minimise the objective
solver.Minimize(solver.Sum(objective_terms))

# solve
status = solver.Solve()

# create dictionary to map integers to partition name
part = {0:"train",1:"development",2:"test"}

output = [] # for saving the partition allocation per speaker
bad_allocations = [] # for saving all the partitions with high cost

# if the solver was successfull
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    # print out total cost
    print(f'Total cost = {solver.Objective().Value():.2f}\n')

    for i in range(n_S): # for each speaker
        for j in range(n_P): # for each partition
            # test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                # collect the allocation
                print(f'Speaker {i}: {df["speaker"][i]}: assigned to partition {part[j]}.' +
                      f' Cost: {C[i][j]:2f}')
                output.append(j)
                
                # if cost of assigning speaker to partition was 'high' add to bad allocations
                if C[i,j] > 1e3:
                    bad_allocations.append((df["speaker"][i],part[j],f"{C[i][j]:.2f}"))
else:
    # if solver was not successful
    print('No solution found.')

Total cost = 39705.29

Speaker 0: SPEAKER_1: assigned to partition train. Cost: 0.099029
Speaker 1: SPEAKER_2: assigned to partition train. Cost: 0.059866
Speaker 2: SPEAKER_3: assigned to partition train. Cost: 0.005808
Speaker 3: SPEAKER_4: assigned to partition train. Cost: 0.072193
Speaker 4: SPEAKER_5: assigned to partition train. Cost: 0.420206
Speaker 5: SPEAKER_6: assigned to partition train. Cost: 0.018340
Speaker 6: SPEAKER_7: assigned to partition train. Cost: 0.043195
Speaker 7: SPEAKER_8: assigned to partition test. Cost: 2035.539934
Speaker 8: SPEAKER_9: assigned to partition test. Cost: 100.019967
Speaker 9: SPEAKER_10: assigned to partition train. Cost: 0.141527
Speaker 10: SPEAKER_11: assigned to partition train. Cost: 0.066187
Speaker 11: SPEAKER_12: assigned to partition test. Cost: 840.368507
Speaker 12: SPEAKER_13: assigned to partition train. Cost: 0.030461
Speaker 13: SPEAKER_14: assigned to partition train. Cost: 0.044015
Speaker 14: SPEAKER_15: assigned to part

### Lets observe the 'bad' allocations

In [69]:
for s,p,c in (bad_allocations):
    print(f"Assigning {s} to {p} cost: {c}")

Assigning SPEAKER_8 to test cost: 2035.54
Assigning SPEAKER_24 to test cost: 1381.39
Assigning SPEAKER_53 to test cost: 2316.10
Assigning SPEAKER_96 to test cost: 8118.19
Assigning SPEAKER_132 to development cost: 1259.89
Assigning SPEAKER_172 to test cost: 1734.95
Assigning SPEAKER_174 to development cost: 9351.49
Assigning SPEAKER_219 to development cost: 1716.43
Assigning SPEAKER_307 to test cost: 1957.32


In [24]:
df['partition'] =  output # add assignment to dataframe

train_set = df.loc[df.partition == 0] # create sub-dataframe for training set
dev_set = df.loc[df.partition == 1] # create sub-dataframe for development set
test_set = df.loc[df.partition == 2] # create sub-dataframe for test set

train_set.to_csv("splits/train.csv",index=False)
dev_set.to_csv("splits/dev.csv",index=False)
test_set.to_csv("splits/test.csv",index=False)

## Show language duration stats of resulting partitions

In [25]:
for n,s in [("Training",train_set),("Development",dev_set),("Test",test_set)]:
    print(16 * '-')
    print(f"{n} set:")
    print(16 * '-')
    print(f"ENG: {s.sum(axis=0)['ENG'] / 60:.2f}")
    print(f"ZUL: {s.sum(axis=0)['ZUL'] / 60:.2f}")
    print(f"XHO: {s.sum(axis=0)['XHO'] / 60:.2f}")
    print(f"SOT: {s.sum(axis=0)['SOT'] / 60:.2f}")
    print(f"TSN: {s.sum(axis=0)['TSN'] / 60:.2f}")
    print(f"ENG-ZUL: {s.sum(axis=0)['ENG-ZUL'] / 60:.2f}")
    print(f"ENG-XHO: {s.sum(axis=0)['ENG-XHO'] / 60:.2f}")
    print(f"ENG-SOT: {s.sum(axis=0)['ENG-SOT'] / 60:.2f}")
    print(f"ENG-TSN: {s.sum(axis=0)['ENG-TSN'] / 60:.2f}")
    print(f"ENG-NSO: {s.sum(axis=0)['ENG-NSO'] / 60:.2f}")
    print(f"AFR-ENG: {s.sum(axis=0)['AFR-ENG'] / 60:.2f}")

----------------
Training set:
----------------
ENG: 655.47
ZUL: 387.25
XHO: 46.52
SOT: 88.34
TSN: 109.76
ENG-ZUL: 409.13
ENG-XHO: 27.28
ENG-SOT: 206.12
ENG-TSN: 126.83
ENG-NSO: 6.54
AFR-ENG: 1.22
----------------
Development set:
----------------
ENG: 202.47
ZUL: 2.80
XHO: 14.03
SOT: 6.32
TSN: 4.77
ENG-ZUL: 15.24
ENG-XHO: 15.20
ENG-SOT: 17.38
ENG-TSN: 15.10
ENG-NSO: 0.05
AFR-ENG: 0.17
----------------
Test set:
----------------
ENG: 240.04
ZUL: 17.41
XHO: 27.24
SOT: 11.07
TSN: 13.43
ENG-ZUL: 50.82
ENG-XHO: 35.48
ENG-SOT: 50.04
ENG-TSN: 50.24
ENG-NSO: 15.77
AFR-ENG: 1.30
